In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout
import matplotlib.pyplot as plt
import tensorflow as tf

In [3]:
df = pd.read_excel("/content/drive/MyDrive/Colab Notebooks/Fiverr/vkthe1st/Time series dataset.xlsx")
df

,Time,Data points
0,1,211090
1,2,336722
2,3,203122
3,4,147749
4,5,297089
...,...,...
195,196,368131
196,197,2779
197,198,338822
198,199,149676


In [4]:
# Normalize the data
scaler = MinMaxScaler()
df['Data points'] = scaler.fit_transform(df['Data points'].values.reshape(-1, 1))

In [5]:
df

,Time,Data points
0,1,0.563095
1,2,0.902362
2,3,0.541578
3,4,0.392044
4,5,0.795334
...,...,...
195,196,0.987181
196,197,0.000556
197,198,0.908033
198,199,0.397248


In [6]:
# Prepare the data for LSTM
sequence_length = 10  # You can adjust this based on your needs
X, y = [], []
for i in range(len(df) - sequence_length):
    X.append(df['Data points'].values[i:i+sequence_length])
    y.append(df['Data points'].values[i+sequence_length])

X = np.array(X)
y = np.array(y)

In [7]:
# Reshape for LSTM input
X = X.reshape(X.shape[0], X.shape[1], 1)

In [8]:

# Build a more complex LSTM model
model = Sequential()
model.add(LSTM(units=200, activation='relu', return_sequences=True, input_shape=(X.shape[1], 1)))
model.add(Dropout(0.2))  # Adding dropout for regularization
model.add(LSTM(units=100, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=1))

model.compile(optimizer='adam', loss='mean_absolute_error')


In [9]:
# Train the model
model.fit(X, y, epochs=10000, batch_size=40)

Streaming output truncated to the last 5000 lines.
5/5 [==============================] - 0s 68ms/step - loss: 0.0065
Epoch 7502/10000
5/5 [==============================] - 0s 63ms/step - loss: 0.0059
Epoch 7503/10000
5/5 [==============================] - 0s 71ms/step - loss: 0.0075
Epoch 7504/10000
5/5 [==============================] - 0s 72ms/step - loss: 0.0068
Epoch 7505/10000
5/5 [==============================] - 0s 70ms/step - loss: 0.0060
Epoch 7506/10000
5/5 [==============================] - 0s 66ms/step - loss: 0.0060
Epoch 7507/10000
5/5 [==============================] - 0s 68ms/step - loss: 0.0069
Epoch 7508/10000
5/5 [==============================] - 0s 65ms/step - loss: 0.0065
Epoch 7509/10000
5/5 [==============================] - 0s 67ms/step - loss: 0.0076
Epoch 7510/10000
5/5 [==============================] - 0s 72ms/step - loss: 0.0061
Epoch 7511/10000
5/5 [==============================] - 0s 65ms/step - loss: 0.0064
Epoch 7512/10000
5/5 [====================

Forecasting

In [10]:
# Forecast for the last 10 data points
forecasted_values = []

sequence_length = 10

for i in range(len(df) - sequence_length, len(df)):
    last_10_points = df['Data points'].values[i - sequence_length:i]
    last_10_points = last_10_points.reshape(1, sequence_length, 1)

    # Make predictions
    predicted_value = model.predict(last_10_points)

    # Inverse transform the predicted value
    predicted_value = scaler.inverse_transform(predicted_value.reshape(1, -1))[0][0]

    forecasted_values.append(predicted_value)

1/1 [==============================] - 0s 34ms/step


In [11]:
forecasted_values

[148091.89,
 353010.88,
 136155.17,
 277515.8,
 339717.3,
 361647.28,
 8774.816,
 334897.25,
 149784.0,
 209568.25]

In [12]:
# Inverse transform the actual values for the last 10 data points
actual_values = scaler.inverse_transform(df['Data points'].values[-10:].reshape(-1, 1)).flatten()
actual_values

array([149018., 361409., 136200., 276169., 344519., 368131.,   2779.,
       338822., 149676., 209049.])

In [13]:
# Calculate MAE
mae = np.mean(np.abs(actual_values - forecasted_values))

mae

3254.909765625